# Transformacion de los Datos para Modelado CLTV

Esta estapa es crucial para preparar el dataset para el modelado del CLTV con Redes Neuronales. Esta fase se realiza ya que no se pueden usar los datos tal cual como estan antes de esta etapa, se requiere transformar el registro de transacciones que esta en formato (filas por compras) a un formato de registros por cliente (una fila por cliente) estructurado en ventanas de tiempo.

In [244]:
import pandas as pd
from dateutil.relativedelta import relativedelta

df = pd.read_parquet('../data/processed/online_retail_II_cleaned.parquet')

## Creacion de Nueva Variable en el Dataset 

In [ ]:
df['totalamount'] = df['quantity'] * df['price'] #Se crea la nueva variable totalamount

# Verificación rápida
print("Estadísticas de totalamount:")
print(df['totalamount'].describe())

Estadísticas de totalamount:
count    400916.000000
mean         21.945330
std          77.734238
min           0.001000
25%           5.000000
50%          12.500000
75%          19.500000
max       15818.400391
Name: totalamount, dtype: float64


Se crea la variable `totalamount` que representa el monto total gastado por cada transaccion, calculado como la multiplicacion de las series `quantity` y `price`.

La creacion de esta nueva variable es esencial para el analisis del CLTV, ya que a nivel cliente, el CLTV se calcula en base al monto total gastado por cada cliente en un periodo de tiempo determinado.

In [236]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400916 entries, 0 to 400915
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoice      400916 non-null  string        
 1   stockcode    400916 non-null  string        
 2   description  400916 non-null  string        
 3   quantity     400916 non-null  int16         
 4   invoicedate  400916 non-null  datetime64[ns]
 5   price        400916 non-null  float32       
 6   customer_id  400916 non-null  Int32         
 7   country      400916 non-null  string        
 8   totalamount  400916 non-null  float32       
dtypes: Int32(1), datetime64[ns](1), float32(2), int16(1), string(4)
memory usage: 21.0 MB


## Horizonte de Tiempo y Ventanas Móviles

In [ ]:
ventanas = [] # se define lista para almacenar las ventanas generadas en el bucle


# Solo podemos empezar en meses donde tengamos 9 meses por delante (3 obs + 6 target)
# Bucle para crear las fechas de inicio de cada ventana
fechas_inicio = [pd.Timestamp('2009-12-01') + relativedelta(months=i) for i in range(4)]
fecha_fin = df['invoicedate'].max().normalize()

for i, start_date in enumerate(fechas_inicio):
    # Definir fechas clave para esta ventana
    division_fechas = start_date + relativedelta(months=3)  # Fin de X, Inicio de y
    end_window_date = division_fechas + relativedelta(months=6) # Fin de y

    # Validar que no nos pasemos del final de los datos reales
    if end_window_date > fecha_fin + pd.Timedelta(days=5): # Margen de error pequeño
        print(f"Ventana {i+1} descartada: termina en {end_window_date}, data termina en {fecha_fin}")
        break

    print(f"Procesando Ventana {i+1}: Obs[{start_date.date()} a {division_fechas.date()}) -> Target[{division_fechas.date()} a {end_window_date.date()}]")

    # 1. Filtrar Datos para X (Observación - 3 meses)
    df_X = df[(df['invoicedate'] >= start_date) & (df['invoicedate'] < division_fechas)]

    # 2. Filtrar Datos para y (Target - 6 meses)
    df_y = df[(df['invoicedate'] >= division_fechas) & (df['invoicedate'] <= end_window_date)]

    # 3. Construir Features (X)
    X_window = df_X.groupby('customer_id').agg({
        'invoicedate': lambda x: (division_fechas - x.max()).days, # Recency relativa al corte
        'invoice': 'nunique',
        'totalamount': 'sum',
        'stockcode': 'nunique'
    }).reset_index()
    X_window.columns = ['customer_id', 'recency', 'frequency', 'monetary', 'product_variety']

    # Feature Contextual: Añadimos el mes de inicio para que el modelo sepa la estacionalidad
    X_window['month_start'] = start_date.month

    # 4. Construir Target (y)
    y_window = df_y.groupby('customer_id')['totalamount'].sum().reset_index()
    y_window.columns = ['customer_id', 'target_cltv_6m']

    # 5. Unir y Etiquetar
    window_data = pd.merge(X_window, y_window, on='customer_id', how='left')
    window_data['target_cltv_6m'] = window_data['target_cltv_6m'].fillna(0)
    # Agregar a la lista ventanas antes creada
    ventanas.append(window_data)

# Concatenar todas las ventanas en un solo gran dataset
df_model_rolling = pd.concat(ventanas, ignore_index=True)

print(f"\nDataset Final Generado: {df_model_rolling.shape[0]} registros totales.")
print(f"Ejemplos promedio por cliente: {df_model_rolling.shape[0] / df_model_rolling['customer_id'].nunique():.2f}")
df_model_rolling.head()

Procesando Ventana 1: Obs[2009-12-01 a 2010-03-01) -> Target[2010-03-01 a 2010-09-01]
Procesando Ventana 2: Obs[2010-01-01 a 2010-04-01) -> Target[2010-04-01 a 2010-10-01]
Procesando Ventana 3: Obs[2010-02-01 a 2010-05-01) -> Target[2010-05-01 a 2010-11-01]
Procesando Ventana 4: Obs[2010-03-01 a 2010-06-01) -> Target[2010-06-01 a 2010-12-01]

Dataset Final Generado: 7509 registros totales.
Ejemplos promedio por cliente: 2.78


,customer_id,recency,frequency,monetary,product_variety,month_start,target_cltv_6m
0,12346,37,9,203.500000,2,12,169.360001
1,12358,82,1,1429.829956,17,12,268.100006
2,12359,74,2,838.890015,34,12,1173.140015
3,12360,6,1,118.000000,10,12,662.039978
4,12361,33,1,109.199997,7,12,98.150002


Se utilizan ventanas móviles para capturar la evolución del comportamiento del cliente a lo largo del tiempo. Cada ventana consta de un periodo de observación (X) y un periodo objetivo (y).
- Periodo de Observación (X): 3 meses
- Periodo Objetivo (y): 6 meses
Se crean múltiples ventanas móviles para cubrir diferentes periodos de tiempo en el dataset, permitiendo capturar cambios en el comportamiento del cliente.
#### Generacion de Ventanas Móviles
Se generan ventanas móviles utilizando un bucle que itera sobre las fechas de inicio definidas. Para cada ventana, se definen las fechas clave para el periodo de observación y el periodo objetivo, y se extraen las características (X) y la variable objetivo (y) correspondientes a cada ventana. Finalmente, se combinan las características y la variable objetivo en un solo dataset para cada ventana, y se almacenan en una lista para su posterior concatenación.

## Pipeline de Transformacion:

In [238]:
import numpy as np
from sklearn.preprocessing import StandardScaler

### 1. Transformacion logarítmica (corregir el sesgo de Pareto)

In [ ]:
# Aplicamos Logaritmo (log(x + 1))
# Esto comprime los valores gigantes (outliers) y ayuda a la red neuronal a converger.
cols_to_log = ['recency', 'frequency', 'monetary', 'product_variety', 'target_cltv_6m']

for col in cols_to_log:
    # No s filtran negativos porque ya no existen en nuestro dataset, anteriormente fue limpiado y validado
    df_model_rolling[col] = np.log1p(df_model_rolling[col])

La transformacion logaritmica se aplica a las variables `recency`, `frequency`, `monetary`, `product_variety` y `target_cltv_6m` utilizando la funcion `log1p` de numpy. Esta transformacion comprime los valores extremos (outliers) y ayuda a la red neuronal a estabilizarse y  obtener mejor eficiencia durante el entrenamiento, reduciendo el sesgo que introducen los outliers.

### 2. Codificación de Estacionalidad (One-Hot Encoding)

In [ ]:
# Convertimos el mes de inicio de la ventana en columnas binarias (categoricas) usando One-Hot Encoding
df_encoded = pd.get_dummies(df_model_rolling, columns=['month_start'], prefix='start_month', dtype=int)

Se convierte la variable categórica `month_start`, que indica el mes de inicio de cada ventana móvil, en variables binarias (one-hot encoding). Esto permite que el modelo capture patrones estacionales en el comportamiento del cliente relacionados con el mes de inicio de la ventana.

### 3. División de Datos en Conjuntos de Entrenamiento y Prueba

In [ ]:
# Garantizamos que un cliente y todas sus ventanas temporales caigan en el mismo grupo
unique_customers = df_encoded['customer_id'].unique()
n_customers = len(unique_customers)

# Semilla para reproducibilidad
np.random.seed(42)
shuffled_ids = np.random.permutation(unique_customers)

# 80% Entrenamiento / 20% Prueba
split_idx = int(n_customers * 0.8)
train_ids = shuffled_ids[:split_idx]
test_ids = shuffled_ids[split_idx:]

# Filtramos el dataFrame
train_df = df_encoded[df_encoded['customer_id'].isin(train_ids)]
test_df = df_encoded[df_encoded['customer_id'].isin(test_ids)]

print(f"Dataset de Entrenamiento: {len(train_ids)} clientes | {len(train_df)} ventanas")
print(f"Dataset de Prueba:        {len(test_ids)} clientes | {len(test_df)} ventanas")

Dataset de Entrenamiento: 2162 clientes | 6042 ventanas
Dataset de Prueba:        541 clientes | 1467 ventanas


La division de los datos se realiza a nivel de cliente para evitar la fuga de datos entre el conjunto de entrenamiento y prueba. Se asegura que todas las ventanas temporales de un mismo cliente caigan en el mismo conjunto, ya sea de entrenamiento o prueba. Esto es crucial para evaluar correctamente el rendimiento del modelo en datos no vistos durante el entrenamiento.

### 4. Estandariazación de los datos

In [ ]:
# Separar features y target
features_col = [c for c in df_encoded.columns if c not in ['customer_id', 'target_cltv_6m']]
target_col = 'target_cltv_6m'

X_train = train_df[features_col].values
y_train = train_df[target_col].values

X_test = test_df[features_col].values
y_test = test_df[target_col].values

# El scaler aprende solo del set de entrenamiento para evitar "Fuga de Datos"
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) # Solo transforma el test, no aprende de él

print(f"\nDimensiones listas para la Red Neuronal:")
print(f"Inputs (X): {X_train_scaled.shape}")


Dimensiones listas para la Red Neuronal:
Inputs (X): (6042, 8)


Se aplica la estandarizacion de los datos utilizando el StandardScaler de sklearn. Este proceso transforma las características para que tengan una media de 0 y una desviación estándar de 1, lo que ayuda a mejorar la convergencia del modelo durante el entrenamiento. El scaler se ajusta únicamente con los datos de entrenamiento para evitar la fuga de datos (data leakage) y luego se aplica tanto al conjunto de entrenamiento como al de prueba.

### 5. Preparación y guardado de datos para entrenamiento de modelo

In [ ]:
import os
import joblib

os.makedirs('../data/processed/', exist_ok=True) #crear carpeta si no existe

print("Guardando archivos en 'data/processed/'...")

# Arrays Matemáticos (Para la Red Neuronal)
joblib.dump(X_train_scaled, '../data/processed/X_train_scaled.pkl', compress=3)
joblib.dump(X_test_scaled, '../data/processed/X_test_scaled.pkl', compress=3)
joblib.dump(y_train, '../data/processed/y_train.pkl', compress=3)
joblib.dump(y_test, '../data/processed/y_test.pkl', compress=3)

# El Scaler (necesario para re-transformar datos en el futuro)
joblib.dump(scaler, '../data/processed/scaler.pkl')

# Datos de Contexto (Para saber que cliente es cada predicción después)
# Guardamos el DF original (con IDs) y las listas de IDs de train/test
df_model_rolling.to_parquet('../data/processed/df_model_rolling.parquet')
joblib.dump(train_ids, '../data/processed/train_ids.pkl')
joblib.dump(test_ids, '../data/processed/test_ids.pkl')

# Guardar los nombres de las columnas después del One-Hot Encoding
joblib.dump(features_col, '../data/processed/model_columns.pkl') #vital para alinear las columnas en producción

print("Feature Engineering finalizado")

Guardando archivos en 'data/processed/'...
¡Proceso de Feature Engineering finalizado y guardado!
